In [ ]:
# Snells law in vector form

import math
import numpy as np
import matplotlib.pyplot as plt
import torch

from torchlensmaker.raytracing import rot2d, rot2d_matrix, super_refraction


def plotv(v, **kwargs):
    plt.plot([0, v[0]], [0, v[1]], **kwargs)
    

def demo_super_refraction(refraction_function, theta_i, normal_angle, n1, n2):
    
    if n1 > n2:
        critical_angle = np.arcsin(n2/n1)
        print("Critical angle", np.degrees(np.arcsin(n2/n1)))
    else:
        print("No critical angle")
    
    normal = rot2d(np.array([1, 0]), normal_angle)
    R = -rot2d(normal, theta_i)
    
    assert(np.allclose(np.linalg.norm(R), 1.0))
    assert(np.allclose(np.linalg.norm(normal), 1.0))
    assert( np.allclose(-R.dot(normal), np.cos(theta_i)) )
    
    R_refracted = refraction_function(torch.tensor(R), torch.tensor(normal), n1, n2)
    
    theta_r = np.arctan2(R_refracted[1], R_refracted[0]) - np.arctan2(-normal[1], -normal[0])
    
    print("theta_i", np.degrees(theta_i))
    print("theta_r", np.degrees(theta_r))
    
    correct_refraction = np.allclose( n1*np.sin(theta_i), n2*np.sin(theta_r) )
    assert(np.allclose(np.linalg.norm(R_refracted), 1.0))

    plotv(rot2d(normal, np.pi/2), color="lightblue")
    plotv(rot2d(normal, -np.pi/2), color="lightblue")
    plotv(normal, linestyle="--", color="grey")
    plotv(-normal, linestyle="--", color="grey")
    plotv(-R, color="orange")

    plotv(R_refracted, color="red")

    plt.gca().set_xlim([-1, 1])
    plt.gca().set_ylim([-1, 1])
    plt.gca().set_title("Normal refraction" if correct_refraction else "Modified refraction!")
    plt.gca().set_aspect("equal")
    plt.show()

def plot_super_refraction():
    n1 = 1.49
    n2 = 1.0
    N_rays = 200

    normal = np.array([0., 1.])
    theta_i = np.linspace(-np.pi/2, np.pi/2, N_rays)
    theta_r = np.zeros_like(theta_i)
    super_refract = np.zeros_like(theta_i)
    
    for i, ti in enumerate(theta_i):
        incident_ray = np.matmul(-normal, rot2d_matrix(ti))
        refracted_ray = super_refraction(torch.tensor(incident_ray), torch.tensor(normal), n1, n2)

        theta_r[i] = np.arctan2(refracted_ray[1], refracted_ray[0]) - np.arctan2(-normal[1], -normal[0])

    plt.plot(np.degrees(theta_i), np.degrees(theta_r))
    plt.gca().set_xlim([-math.pi/2, math.pi/2])
    plt.gca().set_aspect("equal")

plt.figure()
demo_super_refraction(super_refraction, theta_i = np.radians(42.16), normal_angle=np.radians(105), n1=1.49, n2=1.0)

plt.figure()
plot_super_refraction()

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

from torchlensmaker.raytracing import super_refraction, clamped_refraction

def plot(theta_i, normal_angle, n1, n2):
    demo_super_refraction(super_refraction, np.radians(theta_i), np.radians(normal_angle), n1, n2)
    demo_super_refraction(clamped_refraction, np.radians(theta_i), np.radians(normal_angle), n1, n2)

plot(
    theta_i = 42.16,
    normal_angle = 105,
    n1 = 1.0,
    n2 = 1.5,
)